In [13]:
%matplotlib inline
import pandas as pd
import numpy as np
import yfinance as yf
import math
from pandas_datareader import data as pdr
import cufflinks as cf
import plotly as py
import matplotlib as plt
import plotly.express as px
yf.pdr_override()
cf.go_offline()
from plotly.subplots import make_subplots

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as wg
from IPython.display import display, clear_output
import tabulate as tb
from IPython.core.display import HTML
import plotly.graph_objects as go

In [14]:
start_date = "2017-01-01"
end_date = "2018-01-01"
trading_days = 253
riskfree = 0.01

portfolio_dict = {}

# Formula for calculating the Sharpe Ratio
def sharpeRatio(riskfree, stockreturn, std):
    return (stockreturn-riskfree)/std

class Portfolio:
    def __init__(self, name, assets, weights):
        self.name = name
        self.assets = assets
        self.weights = pd.Series(weights)
        
#         Retrieving financial data from Yahoo Finance
        self.stock_data = pdr.get_data_yahoo(assets, start = start_date, end = end_date)['Adj Close']
        
#         caclulating average returns
        self.daily_returns = self.stock_data.pct_change()
        
#         Calculating logarithmic returns
        self.log_return = np.log(self.stock_data/self.stock_data.shift(1))
        self.log_portfolio = np.dot(self.weights.T, self.log_return.mean()) * trading_days
        
        self.port_logreturn = self.log_return * np.array(self.weights)
        
#         calculating variance of portfolio
        self.covariance = self.log_return.cov() * trading_days
        self.portfolio_var = np.dot(self.weights.T, np.dot(self.covariance, self.weights))

#         calculating sharpe ratio of portfolio
        self.sharpe_ratio = sharpeRatio(riskfree, self.log_portfolio, math.sqrt(self.portfolio_var))

In [15]:
# Testing different portfolios
p1 = Portfolio('p1', ['AAPL', 'MSFT', 'BAC'], [0.34, 0.16, 0.5])
p2 = Portfolio('p2', ['GOOG', 'TSLA'], [0.6, 0.4])
p3 = Portfolio('p3', ['XOM', 'BUD', 'FB', 'ORCL', 'INTC'], [0.20, 0.20, 0.20, 0.20, 0.20])
portfolio_dict['p1'] = p1
portfolio_dict['p2'] = p2
portfolio_dict['p3'] = p3

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  5 of 5 completed


In [16]:
# Set market values
SP500 = pdr.get_data_yahoo('^SP500TR', start = start_date, end = end_date)['Adj Close']
market_dailyreturns = np.log(SP500/SP500.shift(1))
market_var = market_dailyreturns.var() * trading_days
market_return = market_dailyreturns.mean() * trading_days

# function for calculating the beta of an portfolio
def calculateBeta(portfolio):
    df = pd.DataFrame({'market': market_dailyreturns,
                        portfolio.name: portfolio.port_logreturn.sum(axis=1)})
    cov = df.cov().iloc[0][1] * trading_days
    beta = cov/market_var
    return beta

# calculating the return according to the capm model
def capm(beta):
    ret = beta * (market_return - riskfree) + riskfree
    return ret

# plot the capm model
def capmPlot():
    x = np.arange(0, 2, 0.01)
    y = np.array([capm(i) for i in x])
    d = pd.DataFrame({'x': x,
                 'y': y})
    
    trace0 = go.Scatter(x=d['x'], y=d['y'], mode = 'lines', name='CAPM')
    trace1 = go.Scatter(x=[1], y=[market_return], mode='markers', name='market portfolio')
    data = [trace0, trace1]
    for name in portfolio_dict.keys():
        portfolio = portfolio_dict.get(name)
        data.append(go.Scatter(x=[calculateBeta(portfolio)], y=[portfolio.log_portfolio], mode = 'markers', name=portfolio.name))
        
    layout = go.Layout(title = 'CAPM model')
    figure = go.Figure(data=data, layout = layout)
    figure.show()

[*********************100%***********************]  1 of 1 completed


In [17]:
# function for giving the stats of all portfolios
def overviewPortfoliosStats():
    df = pd.DataFrame(columns = ['Portfolio', 'Return', 'Volatility', 'Sharpe Ratio'])
    
    for name in portfolio_dict.keys():
        portfolio = portfolio_dict.get(name)
        df = df.append({'Portfolio': portfolio.name,
                   'Return': portfolio.log_portfolio,
                   'Volatility': np.sqrt(portfolio.portfolio_var),
                   'Sharpe Ratio': portfolio.sharpe_ratio}, ignore_index = True)
    df = df.set_index('Portfolio')
    return df

# function for printing the portfolio weights
def overviewPortfolios():
    for name in portfolio_dict.keys():
        series_list = []
        portfolio = portfolio_dict.get(name)
        df = pd.DataFrame({'Company': portfolio.assets,
                           'Weights': portfolio.weights})
        print("Portfolio: " + name)
        print(df)
        print("\n")

In [18]:
# function or making a dataframe with the portfolio stocks
def indexDataframe(name):
    indexOverview = pd.DataFrame()
    portfolio = portfolio_dict.get(name)
    
    
    indexOverview = portfolio.stock_data/portfolio.stock_data.iloc[0]
    indexOverview['S&P 500'] = SP500/SP500[0]
    indexOverview['Portfolio'] = indexOverview.sum(axis = 1) - indexOverview['S&P 500']
    indexOverview['Portfolio'] = indexOverview['Portfolio']/indexOverview['Portfolio'].iloc[0]
    indexOverview = indexOverview * 100
    return indexOverview

# plot the dataframe with indices of portfolio stocks
def indexDataframePlot(name):
    indexDataframe(name).iplot(title='Stock Index Comparison')

In [19]:
# function for creating a daframe for comparing the indices of portfolios
def comparePortfolios():
    df = pd.DataFrame()
    df['S&P 500'] = (SP500/SP500[0])*100
    for name in portfolio_dict.keys():
        portfolio = portfolio_dict.get(name)
        df[name] = indexDataframe(name)['Portfolio']
    return df

# plot the dataframe with portfolio indices
def comparePortfoliosPlot():
    comparePortfolios().iplot(title='Portfolio Index Comparison')

In [20]:

def weightOpt(name, nr_randomnumbers, riskfree):
    
    portfolio = portfolio_dict.get(name)
    np.random.seed(1)

    df_random = pd.DataFrame(np.random.random(size = (len(portfolio.stock_data.T), nr_randomnumbers)))
    df_random = df_random/df_random.sum()
    data = portfolio.stock_data
    stock_returns = np.log(data/data.shift(1))
    expected_returns = np.dot(df_random.T * trading_days, stock_returns.mean() ) 
    
    
    
    coveriances = stock_returns.cov()
    variances = np.zeros(nr_randomnumbers)
    sharpe_ratios = np.zeros(nr_randomnumbers)
    
    for i in range(nr_randomnumbers):
        variances[i] = np.dot(df_random[i], np.dot(coveriances *trading_days, df_random[i]))
        sharpe_ratios[i] = sharpeRatio(riskfree, expected_returns[i], np.sqrt(variances[i]))
    df = pd.DataFrame({'Return': expected_returns,
                        'Volatility': np.sqrt(variances),
                        'Sharpe Ratio': sharpe_ratios})
    
    max = df['Sharpe Ratio'].idxmax(axis = 0)
    opt_weights = df_random.T.iloc[max]
    df_weights = pd.DataFrame({'Company': portfolio.assets,
                                'Weight': opt_weights})
    
    
    print('Optimal Portfolio: ')
    print(tb.tabulate(df_weights, headers = ['Nr', 'Company', 'Weight']))
    
    opt_results = df.iloc[max]
    print("\n")
    print(opt_results)

    scatter = go.Scatter(x=df['Volatility'], y=df['Return'], name='Portfolios', mode='markers', marker=dict(color=df['Sharpe Ratio'], colorbar = dict(title='Sharpe Ratio', x=-0.2)))
    point1 = go.Scatter(x=[df['Volatility'].iloc[max]], y=[df['Return'].iloc[max]], mode='markers', name='Optimal', marker=dict(color='red', size=10))
    data = [scatter, point1]
    
    layout = go.Layout(title = 'Efficient Frontier')
    figure = go.Figure(data=data, layout = layout)
    figure.show()

<h1>Portfolio Analysis Tools</h1>

In [21]:

# method for adding a new portfolio
def addPortfolio(name, assets, weights):
    assets = assets.split(', ')
    weights = [float(weight) for weight in weights.split(', ')]
    portfolio_dict[name] = Portfolio(name, assets, weights)   
    overviewPortfolios()
    
# method for resetting list of portfolios
def resetPortfolio(x):
    portfolio_dict.clear()
    with out: 
        clear_output()
        print('Portfolio list has been reset')

wg.interact_manual(addPortfolio, name = 'TechPortfolio', assets = 'AAPL, MSFT', weights = '0.50, 0.50')

button = wg.Button(description="Reset")
out = wg.Output()
button.on_click(resetPortfolio)


wg.VBox([button, out])


interactive(children=(Text(value='TechPortfolio', description='name'), Text(value='AAPL, MSFT', description='a…

In [22]:
wg.interact_manual(indexDataframePlot, name=portfolio_dict.keys())

interactive(children=(Dropdown(description='name', options=('p1', 'p2', 'p3', 'TechPortfolio'), value='p1'), B…

<function __main__.indexDataframePlot(name)>

In [23]:
overviewPortfolios()
print(tb.tabulate(overviewPortfoliosStats(), headers = ['Name', 'Return', 'Volatility', 'Sharpe Ratio']))

comparePortfoliosPlot()

Portfolio: p1
  Company  Weights
0    AAPL     0.34
1    MSFT     0.16
2     BAC     0.50


Portfolio: p2
  Company  Weights
0    GOOG      0.6
1    TSLA      0.4


Portfolio: p3
  Company  Weights
0     XOM      0.2
1     BUD      0.2
2      FB      0.2
3    ORCL      0.2
4    INTC      0.2


Portfolio: TechPortfolio
  Company  Weights
0    AAPL      0.5
1    MSFT      0.5


Name             Return    Volatility    Sharpe Ratio
-------------  --------  ------------  --------------
p1             0.350604     0.126117          2.70069
p2             0.319804     0.18697           1.65698
p3             0.190837     0.0892383         2.02645
TechPortfolio  0.367837     0.137626          2.60007


In [24]:
capmPlot()

In [12]:
wg.interact_manual(weightOpt, name=portfolio_dict.keys(), nr_randomnumbers = wg.IntSlider(min=10, max=6000, description='Nr Trials', value = 2000), riskfree = wg.FloatSlider(min=0, max=0.20, description='Risk-free rate', value = 0.01, step = 0.0001))

interactive(children=(Dropdown(description='name', options=('p1', 'p2', 'p3', 'TechPortfolio'), value='p1'), I…

<function __main__.weightOpt(name, nr_randomnumbers, riskfree)>